In [34]:
import plotly
from plotly import graph_objs as go
import pyecharts
import pandas as pd
import numpy as np
import copy

In [2]:
import os
project_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('.')), '../'))
import sys
sys.path.append(project_path)
from util.data_process import DataProcess

In [3]:
import os
import xgboost as xgb
from util.data_process import DataProcess as dtp

In [4]:
def evaluate(d_test, pred, silent=1):
    new_d_test = d_test.copy()
    new_d_test['pred'] = pred
    new_d_test['err'] = new_d_test.apply(lambda row: row.price - row.pred, axis=1)
    new_d_test['abserr'] = new_d_test.err.map(lambda x: abs(x))
    new_d_test['ape'] = new_d_test.apply(lambda row: row.abserr / row.price, axis=1)
    mape = np.mean(new_d_test.ape)
    accuracy5p = len(list(filter(lambda x: x <= 0.05, new_d_test.ape))) / len(new_d_test)
    mse = np.mean(new_d_test.err * new_d_test.err)
    rmse = np.sqrt(mse)
    if silent:
        print(f'5%: {round(accuracy5p, 4)*100}%')
        print(f'mse: {round(mse, 4)}')
        print(f'MAPE: {round(mape * 100, 2)}%')
        print(f'rmse: {round(rmse, 4)}')
    return rmse
def gen_line_plotly(img_path, df, attrs=None, names=None, consult_cols=None, **kwargs):
    if isinstance(df, dict):
        df = pd.DataFrame(df)
    keys = list(df.columns) if names is None else names
    if attrs is None:
        attrs = [f'att{i}' for i in range(len(df))]
    elif len(attrs) != len(df):
        raise Exception('attrs length error.')
    else:
        attrs = [f'_{i}' for i in attrs]
    ####
    data = [go.Scatter(x=attrs, y=df[keys[i]], name=keys[i]) for i in range(len(keys))]
    if consult_cols:
        data += [go.Scatter(x=attrs, y=line, name='-', line={'dash': 'dot'}) for line in consult_cols]
    plotly.offline.plot({
        "data": data,
        "layout": go.Layout(title=kwargs.get('graph_name', '-'))
    },
        filename=img_path,
        auto_open=kwargs.get('auto_open', False)
    )


In [5]:
data_train = pd.read_csv(f'{project_path}/data/small_car_price_train.201908.csv')
data_test = pd.read_csv(f'{project_path}/data/small_car_price_test.201908.csv')

In [6]:
with open('car_price_feat.txt') as f:
    feat_list = list(filter(lambda x: x[0] != '#', f.read().split('\n')))

label_encode_map, f_map = DataProcess.gencode(pd.concat([data_train, data_test]), feat_list)
en_train, en_test = DataProcess.encode_process(data_train[feat_list], feat_list, label_encode_map), DataProcess.encode_process(data_test[feat_list], feat_list, label_encode_map)



In [ ]:
train_df = xgb.DMatrix(en_train[feat_list], label=data_train.price)
test_df = xgb.DMatrix(en_test[feat_list], label=data_test.price)
###
evals = [(train_df, 'train'), (test_df, 'test')]

In [63]:
general_params = {
        'booster': 'gbtree',  # gbtree, gblinear, dart
        'silent': 1,  #
        'verbosity': 0,  # 0 (silent), 1 (warning), 2 (info), 3 (debug)
        # 'nthread': 2,
        'disable_default_eval_metric': 0,  # Set to >0 to disable.
    }
tree_params = {
        'eta': 0.045,  # learning_rate, default 0.3
        'gamma': 0.15,  # min_split_loss  越大越保守
        'max_depth': 3,
        'min_child_weight': 3,  # 越大越保守, 叶子节点上所有样本的权重和小于min_child_weight则停止分裂
        # 'max_delta_step': 0,  # it might help in logistic regression when class is extremely imbalanced. Set it to value of 1-10 might help control the update
        'subsample': 0.9,
        'colsample_bytree': 0.9,
        'colsample_bylevel': 1,
        'colsample_bynode': 1,
        'lambda': 1.44,  # L2
        'alpha': 0.09,
        'tree_method': 'hist',  # [auto, exact, approx, hist, gpu_hist]
        # 'sketch_eps': 0.03,  # Only used for tree_method=approx.
        # 'scale_pos_weight': 1,  # sum(negative instances) / sum(positive instances)
        'grow_policy': 'lossguide',  # [depthwise, lossguide] only if tree_method is set to hist
        'max_leaves': 0,  # Only relevant when grow_policy=lossguide is set
        'max_bin': 210,  # Only used if tree_method is set to hist. 直方图数目, 越大越容易过拟合
    }
task_params = {
        'objective': 'reg:linear',  # 'reg:linear',
        'eval_metric': 'rmse',  # rmse for regression, and error for classification, mean average precision for ranking

    }
xgb_params = {**general_params, **tree_params, **task_params}

In [46]:
### alpha l1
### 0.09
rmse_arr = {}
for i in range(80, 100):
    v = i /1000
    d = copy.deepcopy(xgb_params)
    d['alpha'] = v
    bst = xgb.train(d, train_df, 215, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./alpha1.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:25.777	test-rmse:25.6085
[1]	train-rmse:24.3601	test-rmse:24.2205
[2]	train-rmse:23.0235	test-rmse:22.9094
[3]	train-rmse:21.7603	test-rmse:21.7033
[4]	train-rmse:20.5663	test-rmse:20.5375
[5]	train-rmse:19.4371	test-rmse:19.4368
[6]	train-rmse:18.3746	test-rmse:18.3879
[7]	train-rmse:17.3714	test-rmse:17.4129
[8]	train-rmse:16.4247	test-rmse:16.4918
[9]	train-rmse:15.5328	test-rmse:15.6396
[10]	train-rmse:14.6887	test-rmse:14.8177
[11]	train-rmse:13.8945	test-rmse:14.0491
[12]	train-rmse:13.1455	test-rmse:13.3219
[13]	train-rmse:12.4374	test-rmse:12.6319
[14]	train-rmse:11.7702	test-rmse:11.9722
[15]	train-rmse:11.1418	test-rmse:11.3645
[16]	train-rmse:10.5493	test-rmse:10.7817
[17]	train-rmse:9.98711	test-rmse:10.2362
[18]	train-rmse:9.45857	test-rmse:9.72448
[19]	train-rmse:8.95992	test-rmse:9.23844
[20]	train-rmse:8.49108	test-rmse:8.78189
[21]	train-rmse:8.05341	test-rmse:8.35258
[22]	train-rmse:7.63706	test-rmse:7.94768
[23]	train-rmse:7.24737	test-rmse:7.57007
[24

[194]	train-rmse:1.58316	test-rmse:2.02617
[195]	train-rmse:1.58115	test-rmse:2.02555
[196]	train-rmse:1.57987	test-rmse:2.02519
[197]	train-rmse:1.57926	test-rmse:2.02477
[198]	train-rmse:1.57836	test-rmse:2.02385
[199]	train-rmse:1.57763	test-rmse:2.02426
[200]	train-rmse:1.57615	test-rmse:2.02359
[201]	train-rmse:1.57483	test-rmse:2.02382
[202]	train-rmse:1.5742	test-rmse:2.02322
[203]	train-rmse:1.57369	test-rmse:2.0232
[204]	train-rmse:1.57187	test-rmse:2.02219
[205]	train-rmse:1.57066	test-rmse:2.02256
[206]	train-rmse:1.56962	test-rmse:2.02236
[207]	train-rmse:1.56904	test-rmse:2.02202
[208]	train-rmse:1.5677	test-rmse:2.02316
[209]	train-rmse:1.56663	test-rmse:2.02194
[210]	train-rmse:1.56548	test-rmse:2.02282
[211]	train-rmse:1.56515	test-rmse:2.02201
[212]	train-rmse:1.56384	test-rmse:2.02181
[213]	train-rmse:1.56247	test-rmse:2.02144
[214]	train-rmse:1.56126	test-rmse:2.02152
5%: 54.11%
mse: 4.0865
MAPE: 5.72%
rmse: 2.0215
[0]	train-rmse:25.777	test-rmse:25.6085
[1]	train-rm

[172]	train-rmse:1.60958	test-rmse:2.03146
[173]	train-rmse:1.60793	test-rmse:2.03169
[174]	train-rmse:1.60668	test-rmse:2.03221
[175]	train-rmse:1.60477	test-rmse:2.03296
[176]	train-rmse:1.60347	test-rmse:2.03315
[177]	train-rmse:1.60222	test-rmse:2.03204
[178]	train-rmse:1.60186	test-rmse:2.03175
[179]	train-rmse:1.60083	test-rmse:2.03013
[180]	train-rmse:1.60037	test-rmse:2.03028
[181]	train-rmse:1.59888	test-rmse:2.02902
[182]	train-rmse:1.59658	test-rmse:2.02874
[183]	train-rmse:1.59562	test-rmse:2.02736
[184]	train-rmse:1.59428	test-rmse:2.02759
[185]	train-rmse:1.59284	test-rmse:2.02835
[186]	train-rmse:1.59159	test-rmse:2.02873
[187]	train-rmse:1.5898	test-rmse:2.02916
[188]	train-rmse:1.58856	test-rmse:2.02836
[189]	train-rmse:1.58708	test-rmse:2.02844
[190]	train-rmse:1.58596	test-rmse:2.02697
[191]	train-rmse:1.58556	test-rmse:2.02677
[192]	train-rmse:1.58505	test-rmse:2.02659
[193]	train-rmse:1.58434	test-rmse:2.02567
[194]	train-rmse:1.58316	test-rmse:2.02617
[195]	train-

[150]	train-rmse:1.64312	test-rmse:2.04578
[151]	train-rmse:1.64148	test-rmse:2.04429
[152]	train-rmse:1.64067	test-rmse:2.04318
[153]	train-rmse:1.63972	test-rmse:2.0409
[154]	train-rmse:1.63736	test-rmse:2.0384
[155]	train-rmse:1.63565	test-rmse:2.03776
[156]	train-rmse:1.63446	test-rmse:2.03711
[157]	train-rmse:1.63348	test-rmse:2.03719
[158]	train-rmse:1.63215	test-rmse:2.03796
[159]	train-rmse:1.63048	test-rmse:2.03854
[160]	train-rmse:1.62832	test-rmse:2.03883
[161]	train-rmse:1.6263	test-rmse:2.03807
[162]	train-rmse:1.62459	test-rmse:2.03671
[163]	train-rmse:1.62305	test-rmse:2.03593
[164]	train-rmse:1.62167	test-rmse:2.03508
[165]	train-rmse:1.61995	test-rmse:2.03489
[166]	train-rmse:1.61936	test-rmse:2.03463
[167]	train-rmse:1.61834	test-rmse:2.03177
[168]	train-rmse:1.61649	test-rmse:2.03181
[169]	train-rmse:1.61591	test-rmse:2.03185
[170]	train-rmse:1.61469	test-rmse:2.03219
[171]	train-rmse:1.61304	test-rmse:2.0315
[172]	train-rmse:1.61163	test-rmse:2.0302
[173]	train-rmse

[128]	train-rmse:1.66989	test-rmse:2.06258
[129]	train-rmse:1.66853	test-rmse:2.06028
[130]	train-rmse:1.66776	test-rmse:2.05912
[131]	train-rmse:1.66629	test-rmse:2.05773
[132]	train-rmse:1.6649	test-rmse:2.05661
[133]	train-rmse:1.66364	test-rmse:2.05284
[134]	train-rmse:1.6627	test-rmse:2.05262
[135]	train-rmse:1.66074	test-rmse:2.05413
[136]	train-rmse:1.65935	test-rmse:2.05302
[137]	train-rmse:1.65744	test-rmse:2.05104
[138]	train-rmse:1.65596	test-rmse:2.04856
[139]	train-rmse:1.65537	test-rmse:2.04834
[140]	train-rmse:1.65438	test-rmse:2.04835
[141]	train-rmse:1.65354	test-rmse:2.04792
[142]	train-rmse:1.65276	test-rmse:2.04754
[143]	train-rmse:1.6515	test-rmse:2.04712
[144]	train-rmse:1.64977	test-rmse:2.04735
[145]	train-rmse:1.64779	test-rmse:2.04758
[146]	train-rmse:1.6466	test-rmse:2.0472
[147]	train-rmse:1.64593	test-rmse:2.04655
[148]	train-rmse:1.64505	test-rmse:2.04625
[149]	train-rmse:1.64419	test-rmse:2.04607
[150]	train-rmse:1.64312	test-rmse:2.04578
[151]	train-rmse

...

[39]	train-rmse:3.43415	test-rmse:3.87082
[40]	train-rmse:3.30609	test-rmse:3.74844
[41]	train-rmse:3.18689	test-rmse:3.62474
[42]	train-rmse:3.0751	test-rmse:3.51377
[43]	train-rmse:2.97288	test-rmse:3.41355
[44]	train-rmse:2.87787	test-rmse:3.31933
[45]	train-rmse:2.78896	test-rmse:3.2251
[46]	train-rmse:2.70831	test-rmse:3.14474
[47]	train-rmse:2.63418	test-rmse:3.07347
[48]	train-rmse:2.56351	test-rmse:3.00437
[49]	train-rmse:2.49977	test-rmse:2.9441
[50]	train-rmse:2.44007	test-rmse:2.88452
[51]	train-rmse:2.38594	test-rmse:2.82805
[52]	train-rmse:2.335	test-rmse:2.78079
[53]	train-rmse:2.289	test-rmse:2.73323
[54]	train-rmse:2.24575	test-rmse:2.68478
[55]	train-rmse:2.20704	test-rmse:2.64148
[56]	train-rmse:2.17144	test-rmse:2.59757
[57]	train-rmse:2.13901	test-rmse:2.56371
[58]	train-rmse:2.10862	test-rmse:2.53317
[59]	train-rmse:2.08058	test-rmse:2.50044
[60]	train-rmse:2.05406	test-rmse:2.47533
[61]	train-rmse:2.03051	test-rmse:2.44952
[62]	train-rmse:2.00908	test-rmse:2.4224


[17]	train-rmse:9.98716	test-rmse:10.2363
[18]	train-rmse:9.45862	test-rmse:9.72452
[19]	train-rmse:8.95997	test-rmse:9.23849
[20]	train-rmse:8.49113	test-rmse:8.78193
[21]	train-rmse:8.05346	test-rmse:8.35262
[22]	train-rmse:7.63711	test-rmse:7.94773
[23]	train-rmse:7.24743	test-rmse:7.57012
[24]	train-rmse:6.88151	test-rmse:7.21537
[25]	train-rmse:6.53466	test-rmse:6.87489
[26]	train-rmse:6.20699	test-rmse:6.56397
[27]	train-rmse:5.90229	test-rmse:6.27124
[28]	train-rmse:5.61466	test-rmse:5.98736
[29]	train-rmse:5.34415	test-rmse:5.72487
[30]	train-rmse:5.09247	test-rmse:5.47892
[31]	train-rmse:4.85559	test-rmse:5.24609
[32]	train-rmse:4.63427	test-rmse:5.03239
[33]	train-rmse:4.42602	test-rmse:4.8349
[34]	train-rmse:4.23057	test-rmse:4.64825
[35]	train-rmse:4.04805	test-rmse:4.46635
[36]	train-rmse:3.87728	test-rmse:4.31325
[37]	train-rmse:3.71913	test-rmse:4.16021
[38]	train-rmse:3.57152	test-rmse:4.00963
[39]	train-rmse:3.43415	test-rmse:3.87082
[40]	train-rmse:3.30609	test-rmse:3

[211]	train-rmse:1.56617	test-rmse:2.01521
[212]	train-rmse:1.5649	test-rmse:2.01502
[213]	train-rmse:1.56361	test-rmse:2.01521
[214]	train-rmse:1.5624	test-rmse:2.01531
5%: 53.93%
mse: 4.0615
MAPE: 5.7%
rmse: 2.0153
[0]	train-rmse:25.777	test-rmse:25.6085
[1]	train-rmse:24.3602	test-rmse:24.2205
[2]	train-rmse:23.0235	test-rmse:22.9095
[3]	train-rmse:21.7603	test-rmse:21.7033
[4]	train-rmse:20.5663	test-rmse:20.5375
[5]	train-rmse:19.4371	test-rmse:19.4368
[6]	train-rmse:18.3746	test-rmse:18.3879
[7]	train-rmse:17.3714	test-rmse:17.4129
[8]	train-rmse:16.4247	test-rmse:16.4918
[9]	train-rmse:15.5329	test-rmse:15.6396
[10]	train-rmse:14.6888	test-rmse:14.8178
[11]	train-rmse:13.8945	test-rmse:14.0491
[12]	train-rmse:13.1456	test-rmse:13.3219
[13]	train-rmse:12.4375	test-rmse:12.6319
[14]	train-rmse:11.7703	test-rmse:11.9722
[15]	train-rmse:11.1418	test-rmse:11.3646
[16]	train-rmse:10.5493	test-rmse:10.7818
[17]	train-rmse:9.98717	test-rmse:10.2363
[18]	train-rmse:9.45863	test-rmse:9.72

[189]	train-rmse:1.59108	test-rmse:2.02415
[190]	train-rmse:1.59004	test-rmse:2.02272
[191]	train-rmse:1.58955	test-rmse:2.02278
[192]	train-rmse:1.58906	test-rmse:2.02178
[193]	train-rmse:1.58828	test-rmse:2.02135
[194]	train-rmse:1.5871	test-rmse:2.02183
[195]	train-rmse:1.58516	test-rmse:2.02141
[196]	train-rmse:1.5838	test-rmse:2.02092
[197]	train-rmse:1.5832	test-rmse:2.02038
[198]	train-rmse:1.58232	test-rmse:2.01947
[199]	train-rmse:1.58157	test-rmse:2.01986
[200]	train-rmse:1.58003	test-rmse:2.01912
[201]	train-rmse:1.57861	test-rmse:2.01923
[202]	train-rmse:1.57791	test-rmse:2.01941
[203]	train-rmse:1.57742	test-rmse:2.01938
[204]	train-rmse:1.57554	test-rmse:2.01833
[205]	train-rmse:1.57467	test-rmse:2.0176
[206]	train-rmse:1.57205	test-rmse:2.01774
[207]	train-rmse:1.57147	test-rmse:2.01755
[208]	train-rmse:1.56971	test-rmse:2.01834
[209]	train-rmse:1.56859	test-rmse:2.01668
[210]	train-rmse:1.56707	test-rmse:2.01527
[211]	train-rmse:1.56618	test-rmse:2.01522
[212]	train-rms

[167]	train-rmse:1.61921	test-rmse:2.03389
[168]	train-rmse:1.6173	test-rmse:2.03406
[169]	train-rmse:1.61664	test-rmse:2.0342
[170]	train-rmse:1.61551	test-rmse:2.03453
[171]	train-rmse:1.6139	test-rmse:2.03397
[172]	train-rmse:1.61239	test-rmse:2.03242
[173]	train-rmse:1.61071	test-rmse:2.03263
[174]	train-rmse:1.60945	test-rmse:2.03317
[175]	train-rmse:1.60757	test-rmse:2.03392
[176]	train-rmse:1.60628	test-rmse:2.03417
[177]	train-rmse:1.60548	test-rmse:2.03393
[178]	train-rmse:1.60464	test-rmse:2.03178
[179]	train-rmse:1.60357	test-rmse:2.03041
[180]	train-rmse:1.60309	test-rmse:2.03063
[181]	train-rmse:1.60166	test-rmse:2.02945
[182]	train-rmse:1.60038	test-rmse:2.02916
[183]	train-rmse:1.59946	test-rmse:2.02787
[184]	train-rmse:1.5981	test-rmse:2.02731
[185]	train-rmse:1.59676	test-rmse:2.02739
[186]	train-rmse:1.59549	test-rmse:2.02654
[187]	train-rmse:1.59367	test-rmse:2.0246
[188]	train-rmse:1.59245	test-rmse:2.02386
[189]	train-rmse:1.59109	test-rmse:2.02416
[190]	train-rmse

In [ ]:
### lambda l2
### 1.44
rmse_arr = {}
for i in range(1400, 1500, 5):
    v = i /1000
    d = copy.deepcopy(xgb_params)
    d['lambda'] = v
    bst = xgb.train(d, train_df, 215, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./lambda1.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

In [36]:
### max_leaves 没效果
### 
rmse_arr = {}
for i in range(10, 70, 5):
    v = i 
    d = copy.deepcopy(xgb_params)
    d['max_leaves'] = v
    bst = xgb.train(d, train_df, 215, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./max_leaves.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:25.7743	test-rmse:25.6077
[1]	train-rmse:24.3548	test-rmse:24.2179
[2]	train-rmse:23.0161	test-rmse:22.9387
[3]	train-rmse:21.7507	test-rmse:21.6917
[4]	train-rmse:20.5548	test-rmse:20.5239
[5]	train-rmse:19.424	test-rmse:19.4212
[6]	train-rmse:18.3595	test-rmse:18.3709
[7]	train-rmse:17.3555	test-rmse:17.3936
[8]	train-rmse:16.4081	test-rmse:16.4733
[9]	train-rmse:15.5151	test-rmse:15.6201
[10]	train-rmse:14.6696	test-rmse:14.7982
[11]	train-rmse:13.8741	test-rmse:14.0287
[12]	train-rmse:13.1243	test-rmse:13.3006
[13]	train-rmse:12.4154	test-rmse:12.6116
[14]	train-rmse:11.7481	test-rmse:11.956
[15]	train-rmse:11.1195	test-rmse:11.3477
[16]	train-rmse:10.5264	test-rmse:10.7681
[17]	train-rmse:9.96565	test-rmse:10.2271
[18]	train-rmse:9.43767	test-rmse:9.71858
[19]	train-rmse:8.94004	test-rmse:9.24109
[20]	train-rmse:8.4699	test-rmse:8.7806
[21]	train-rmse:8.03195	test-rmse:8.34891
[22]	train-rmse:7.61654	test-rmse:7.94652
[23]	train-rmse:7.22675	test-rmse:7.5705
[24]	tr

[194]	train-rmse:1.58123	test-rmse:2.01652
[195]	train-rmse:1.57927	test-rmse:2.01611
[196]	train-rmse:1.57793	test-rmse:2.01569
[197]	train-rmse:1.57727	test-rmse:2.01512
[198]	train-rmse:1.57639	test-rmse:2.01447
[199]	train-rmse:1.57559	test-rmse:2.01484
[200]	train-rmse:1.57408	test-rmse:2.01411
[201]	train-rmse:1.57274	test-rmse:2.01491
[202]	train-rmse:1.57202	test-rmse:2.0149
[203]	train-rmse:1.57149	test-rmse:2.01488
[204]	train-rmse:1.57061	test-rmse:2.01476
[205]	train-rmse:1.56973	test-rmse:2.01504
[206]	train-rmse:1.56855	test-rmse:2.01586
[207]	train-rmse:1.56796	test-rmse:2.01598
[208]	train-rmse:1.56658	test-rmse:2.01641
[209]	train-rmse:1.56536	test-rmse:2.01574
[210]	train-rmse:1.56406	test-rmse:2.01584
[211]	train-rmse:1.56319	test-rmse:2.01607
[212]	train-rmse:1.56275	test-rmse:2.01607
[213]	train-rmse:1.56188	test-rmse:2.01614
[214]	train-rmse:1.56079	test-rmse:2.01644
5%: 53.559999999999995%
mse: 4.066
MAPE: 5.73%
rmse: 2.0164
[0]	train-rmse:25.7743	test-rmse:25.60

[172]	train-rmse:1.60801	test-rmse:2.02486
[173]	train-rmse:1.60654	test-rmse:2.02498
[174]	train-rmse:1.60526	test-rmse:2.02553
[175]	train-rmse:1.60367	test-rmse:2.02619
[176]	train-rmse:1.60232	test-rmse:2.02635
[177]	train-rmse:1.60155	test-rmse:2.02619
[178]	train-rmse:1.5995	test-rmse:2.02434
[179]	train-rmse:1.59851	test-rmse:2.02255
[180]	train-rmse:1.59803	test-rmse:2.02276
[181]	train-rmse:1.59655	test-rmse:2.02168
[182]	train-rmse:1.59415	test-rmse:2.02142
[183]	train-rmse:1.59327	test-rmse:2.01982
[184]	train-rmse:1.59204	test-rmse:2.02012
[185]	train-rmse:1.59109	test-rmse:2.01929
[186]	train-rmse:1.59029	test-rmse:2.02012
[187]	train-rmse:1.58846	test-rmse:2.02043
[188]	train-rmse:1.58729	test-rmse:2.01967
[189]	train-rmse:1.58574	test-rmse:2.01902
[190]	train-rmse:1.58475	test-rmse:2.01761
[191]	train-rmse:1.58374	test-rmse:2.01685
[192]	train-rmse:1.58317	test-rmse:2.01666
[193]	train-rmse:1.58236	test-rmse:2.01601
[194]	train-rmse:1.58123	test-rmse:2.01652
[195]	train-

[150]	train-rmse:1.63749	test-rmse:2.04213
[151]	train-rmse:1.63609	test-rmse:2.04192
[152]	train-rmse:1.63565	test-rmse:2.04126
[153]	train-rmse:1.63471	test-rmse:2.03877
[154]	train-rmse:1.63336	test-rmse:2.03559
[155]	train-rmse:1.63128	test-rmse:2.03584
[156]	train-rmse:1.62997	test-rmse:2.03522
[157]	train-rmse:1.62895	test-rmse:2.03529
[158]	train-rmse:1.62748	test-rmse:2.03612
[159]	train-rmse:1.62577	test-rmse:2.03534
[160]	train-rmse:1.62386	test-rmse:2.0356
[161]	train-rmse:1.62231	test-rmse:2.03419
[162]	train-rmse:1.62068	test-rmse:2.03255
[163]	train-rmse:1.61906	test-rmse:2.03172
[164]	train-rmse:1.61756	test-rmse:2.0308
[165]	train-rmse:1.61605	test-rmse:2.03028
[166]	train-rmse:1.61551	test-rmse:2.03006
[167]	train-rmse:1.61448	test-rmse:2.02718
[168]	train-rmse:1.61285	test-rmse:2.02743
[169]	train-rmse:1.61228	test-rmse:2.02756
[170]	train-rmse:1.61103	test-rmse:2.0273
[171]	train-rmse:1.6095	test-rmse:2.02687
[172]	train-rmse:1.60801	test-rmse:2.02486
[173]	train-rms

[128]	train-rmse:1.666	test-rmse:2.05966
[129]	train-rmse:1.66442	test-rmse:2.05716
[130]	train-rmse:1.66346	test-rmse:2.05612
[131]	train-rmse:1.66207	test-rmse:2.05341
[132]	train-rmse:1.65985	test-rmse:2.05238
[133]	train-rmse:1.65865	test-rmse:2.04839
[134]	train-rmse:1.65771	test-rmse:2.04824


KeyboardInterrupt: 

In [64]:
### max_bin
### 210
rmse_arr = {}
for i in range(200, 600, 10):
    v = i 
    xgb_params['max_bin'] = v
    bst = xgb.train(xgb_params, train_df, 524, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./max_bin.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:26.0723	test-rmse:25.8967
[1]	train-rmse:24.9204	test-rmse:24.7681
[2]	train-rmse:23.8211	test-rmse:23.7179
[3]	train-rmse:22.7702	test-rmse:22.6879
[4]	train-rmse:21.765	test-rmse:21.7061
[5]	train-rmse:20.8032	test-rmse:20.7679
[6]	train-rmse:19.8874	test-rmse:19.8653
[7]	train-rmse:19.0128	test-rmse:19.0133
[8]	train-rmse:18.1775	test-rmse:18.1999
[9]	train-rmse:17.3814	test-rmse:17.4382
[10]	train-rmse:16.6201	test-rmse:16.6974
[11]	train-rmse:15.8945	test-rmse:15.9897
[12]	train-rmse:15.2011	test-rmse:15.3154
[13]	train-rmse:14.5381	test-rmse:14.6688
[14]	train-rmse:13.9063	test-rmse:14.0471
[15]	train-rmse:13.3033	test-rmse:13.4616
[16]	train-rmse:12.7279	test-rmse:12.8973
[17]	train-rmse:12.177	test-rmse:12.365
[18]	train-rmse:11.6526	test-rmse:11.8576
[19]	train-rmse:11.1506	test-rmse:11.3709
[20]	train-rmse:10.6711	test-rmse:10.9
[21]	train-rmse:10.2178	test-rmse:10.4616
[22]	train-rmse:9.78222	test-rmse:10.0399
[23]	train-rmse:9.36837	test-rmse:9.64073
[24]	tra

[195]	train-rmse:1.62771	test-rmse:2.0261
[196]	train-rmse:1.6263	test-rmse:2.02562
[197]	train-rmse:1.62565	test-rmse:2.02497
[198]	train-rmse:1.62492	test-rmse:2.02468
[199]	train-rmse:1.62416	test-rmse:2.02499
[200]	train-rmse:1.62268	test-rmse:2.02426
[201]	train-rmse:1.62116	test-rmse:2.02464
[202]	train-rmse:1.62013	test-rmse:2.02385
[203]	train-rmse:1.61964	test-rmse:2.02376
[204]	train-rmse:1.61912	test-rmse:2.02295
[205]	train-rmse:1.61797	test-rmse:2.02297
[206]	train-rmse:1.61705	test-rmse:2.02281
[207]	train-rmse:1.61628	test-rmse:2.02298
[208]	train-rmse:1.61487	test-rmse:2.02352
[209]	train-rmse:1.61369	test-rmse:2.02224
[210]	train-rmse:1.61253	test-rmse:2.02299
[211]	train-rmse:1.6117	test-rmse:2.02272
[212]	train-rmse:1.61128	test-rmse:2.02282
[213]	train-rmse:1.61041	test-rmse:2.02289
[214]	train-rmse:1.60938	test-rmse:2.02189
[215]	train-rmse:1.60813	test-rmse:2.02119
[216]	train-rmse:1.60709	test-rmse:2.0195
[217]	train-rmse:1.60606	test-rmse:2.01841
[218]	train-rms

[387]	train-rmse:1.4587	test-rmse:1.99847
[388]	train-rmse:1.45759	test-rmse:1.9983
[389]	train-rmse:1.45722	test-rmse:1.99788
[390]	train-rmse:1.45612	test-rmse:1.99764
[391]	train-rmse:1.45545	test-rmse:1.99694
[392]	train-rmse:1.45454	test-rmse:1.9971
[393]	train-rmse:1.45405	test-rmse:1.99728
[394]	train-rmse:1.45302	test-rmse:1.9974
[395]	train-rmse:1.45255	test-rmse:1.99739
[396]	train-rmse:1.45145	test-rmse:1.99744
[397]	train-rmse:1.45083	test-rmse:1.99674
[398]	train-rmse:1.44974	test-rmse:1.99662
[399]	train-rmse:1.44912	test-rmse:1.99636
[400]	train-rmse:1.44819	test-rmse:1.99626
[401]	train-rmse:1.44752	test-rmse:1.99666
[402]	train-rmse:1.447	test-rmse:1.9968
[403]	train-rmse:1.44606	test-rmse:1.9964
[404]	train-rmse:1.44553	test-rmse:1.99673
[405]	train-rmse:1.44477	test-rmse:1.99577
[406]	train-rmse:1.44427	test-rmse:1.99576
[407]	train-rmse:1.44404	test-rmse:1.99578
[408]	train-rmse:1.44373	test-rmse:1.99567
[409]	train-rmse:1.44325	test-rmse:1.99628
[410]	train-rmse:1.

[55]	train-rmse:2.90211	test-rmse:3.3612
[56]	train-rmse:2.82971	test-rmse:3.28426
[57]	train-rmse:2.76184	test-rmse:3.21626
[58]	train-rmse:2.69731	test-rmse:3.14715
[59]	train-rmse:2.63656	test-rmse:3.08322
[60]	train-rmse:2.57969	test-rmse:3.03072
[61]	train-rmse:2.52698	test-rmse:2.97584
[62]	train-rmse:2.47645	test-rmse:2.91623
[63]	train-rmse:2.42966	test-rmse:2.86985
[64]	train-rmse:2.38576	test-rmse:2.82542
[65]	train-rmse:2.34535	test-rmse:2.78601
[66]	train-rmse:2.30671	test-rmse:2.74626
[67]	train-rmse:2.27137	test-rmse:2.71251
[68]	train-rmse:2.23846	test-rmse:2.67566
[69]	train-rmse:2.20757	test-rmse:2.64216
[70]	train-rmse:2.17761	test-rmse:2.60471
[71]	train-rmse:2.15153	test-rmse:2.57625
[72]	train-rmse:2.12568	test-rmse:2.55484
[73]	train-rmse:2.10106	test-rmse:2.5247
[74]	train-rmse:2.07833	test-rmse:2.49828
[75]	train-rmse:2.05778	test-rmse:2.4753
[76]	train-rmse:2.03753	test-rmse:2.44953
[77]	train-rmse:2.01971	test-rmse:2.43004
[78]	train-rmse:2.00317	test-rmse:2.4

[248]	train-rmse:1.5797	test-rmse:2.01737
[249]	train-rmse:1.57841	test-rmse:2.01791
[250]	train-rmse:1.57778	test-rmse:2.0172
[251]	train-rmse:1.57717	test-rmse:2.0168
[252]	train-rmse:1.5762	test-rmse:2.01662
[253]	train-rmse:1.57578	test-rmse:2.01553
[254]	train-rmse:1.57523	test-rmse:2.01479
[255]	train-rmse:1.57393	test-rmse:2.01485
[256]	train-rmse:1.57289	test-rmse:2.01511
[257]	train-rmse:1.57217	test-rmse:2.01471
[258]	train-rmse:1.57079	test-rmse:2.01462
[259]	train-rmse:1.57004	test-rmse:2.01338
[260]	train-rmse:1.56885	test-rmse:2.01201
[261]	train-rmse:1.56799	test-rmse:2.01203
[262]	train-rmse:1.56717	test-rmse:2.01214
[263]	train-rmse:1.56627	test-rmse:2.01073
[264]	train-rmse:1.56549	test-rmse:2.01005
[265]	train-rmse:1.56469	test-rmse:2.01034
[266]	train-rmse:1.56369	test-rmse:2.00926
[267]	train-rmse:1.56294	test-rmse:2.00863
[268]	train-rmse:1.56223	test-rmse:2.00728
[269]	train-rmse:1.56131	test-rmse:2.0068
[270]	train-rmse:1.56062	test-rmse:2.00691
[271]	train-rmse

...

[117]	train-rmse:1.74961	test-rmse:2.11424
[118]	train-rmse:1.74657	test-rmse:2.11069
[119]	train-rmse:1.74448	test-rmse:2.10904
[120]	train-rmse:1.74156	test-rmse:2.10835
[121]	train-rmse:1.73948	test-rmse:2.10787
[122]	train-rmse:1.7367	test-rmse:2.10505
[123]	train-rmse:1.73448	test-rmse:2.10159
[124]	train-rmse:1.73257	test-rmse:2.09819
[125]	train-rmse:1.73023	test-rmse:2.09777
[126]	train-rmse:1.72792	test-rmse:2.09755
[127]	train-rmse:1.72509	test-rmse:2.09537
[128]	train-rmse:1.72344	test-rmse:2.09207
[129]	train-rmse:1.72148	test-rmse:2.08887
[130]	train-rmse:1.72014	test-rmse:2.08819
[131]	train-rmse:1.7185	test-rmse:2.08553
[132]	train-rmse:1.7169	test-rmse:2.08373
[133]	train-rmse:1.71535	test-rmse:2.07941
[134]	train-rmse:1.71404	test-rmse:2.07896
[135]	train-rmse:1.71204	test-rmse:2.07984
[136]	train-rmse:1.70965	test-rmse:2.07773
[137]	train-rmse:1.70758	test-rmse:2.07537
[138]	train-rmse:1.70568	test-rmse:2.07404
[139]	train-rmse:1.70441	test-rmse:2.07417
[140]	train-rm

[309]	train-rmse:1.51876	test-rmse:2.00793
[310]	train-rmse:1.51827	test-rmse:2.00844
[311]	train-rmse:1.51781	test-rmse:2.00872
[312]	train-rmse:1.51699	test-rmse:2.00867
[313]	train-rmse:1.51673	test-rmse:2.00856
[314]	train-rmse:1.51606	test-rmse:2.00841
[315]	train-rmse:1.51575	test-rmse:2.00827
[316]	train-rmse:1.51446	test-rmse:2.00851
[317]	train-rmse:1.51343	test-rmse:2.00881
[318]	train-rmse:1.51301	test-rmse:2.00896
[319]	train-rmse:1.51235	test-rmse:2.00852
[320]	train-rmse:1.51136	test-rmse:2.00805
[321]	train-rmse:1.51086	test-rmse:2.00762
[322]	train-rmse:1.51016	test-rmse:2.0075
[323]	train-rmse:1.50916	test-rmse:2.00659
[324]	train-rmse:1.5085	test-rmse:2.00519
[325]	train-rmse:1.50695	test-rmse:2.00493
[326]	train-rmse:1.50565	test-rmse:2.00502
[327]	train-rmse:1.50485	test-rmse:2.00503
[328]	train-rmse:1.50353	test-rmse:2.00512
[329]	train-rmse:1.50249	test-rmse:2.00463
[330]	train-rmse:1.50193	test-rmse:2.00474
[331]	train-rmse:1.50115	test-rmse:2.00472
[332]	train-r

[501]	train-rmse:1.38182	test-rmse:1.99596
[502]	train-rmse:1.38104	test-rmse:1.99579
[503]	train-rmse:1.38071	test-rmse:1.99588
[504]	train-rmse:1.38002	test-rmse:1.99613
[505]	train-rmse:1.37943	test-rmse:1.99639
[506]	train-rmse:1.37849	test-rmse:1.99636
[507]	train-rmse:1.37789	test-rmse:1.9954
[508]	train-rmse:1.37703	test-rmse:1.99528
[509]	train-rmse:1.37686	test-rmse:1.9951
[510]	train-rmse:1.37618	test-rmse:1.9955
[511]	train-rmse:1.37556	test-rmse:1.99539
[512]	train-rmse:1.37488	test-rmse:1.99474
[513]	train-rmse:1.37414	test-rmse:1.99478
[514]	train-rmse:1.37378	test-rmse:1.99515
[515]	train-rmse:1.37291	test-rmse:1.99503
[516]	train-rmse:1.37252	test-rmse:1.99517
[517]	train-rmse:1.37182	test-rmse:1.99491
[518]	train-rmse:1.37111	test-rmse:1.99486
[519]	train-rmse:1.37052	test-rmse:1.9951
[520]	train-rmse:1.36996	test-rmse:1.99545
[521]	train-rmse:1.3694	test-rmse:1.99511
[522]	train-rmse:1.36916	test-rmse:1.99512
[523]	train-rmse:1.36854	test-rmse:1.99486
5%: 54.02%
mse: 

[171]	train-rmse:1.66115	test-rmse:2.04678
[172]	train-rmse:1.65952	test-rmse:2.04649
[173]	train-rmse:1.65806	test-rmse:2.04654
[174]	train-rmse:1.65727	test-rmse:2.04612
[175]	train-rmse:1.65526	test-rmse:2.04649
[176]	train-rmse:1.6541	test-rmse:2.04617
[177]	train-rmse:1.65359	test-rmse:2.04615
[178]	train-rmse:1.65156	test-rmse:2.04427
[179]	train-rmse:1.6505	test-rmse:2.04208
[180]	train-rmse:1.64946	test-rmse:2.04183
[181]	train-rmse:1.64788	test-rmse:2.04033
[182]	train-rmse:1.64683	test-rmse:2.03971
[183]	train-rmse:1.64587	test-rmse:2.03965
[184]	train-rmse:1.64421	test-rmse:2.0389
[185]	train-rmse:1.64298	test-rmse:2.03823
[186]	train-rmse:1.64169	test-rmse:2.03856
[187]	train-rmse:1.63998	test-rmse:2.03936
[188]	train-rmse:1.6388	test-rmse:2.03858
[189]	train-rmse:1.63714	test-rmse:2.03861
[190]	train-rmse:1.63638	test-rmse:2.03646
[191]	train-rmse:1.63556	test-rmse:2.03596
[192]	train-rmse:1.63477	test-rmse:2.03569
[193]	train-rmse:1.63397	test-rmse:2.03507
[194]	train-rms

[363]	train-rmse:1.47767	test-rmse:2.00767
[364]	train-rmse:1.47711	test-rmse:2.00707
[365]	train-rmse:1.47648	test-rmse:2.00677
[366]	train-rmse:1.47631	test-rmse:2.00689
[367]	train-rmse:1.47549	test-rmse:2.0071
[368]	train-rmse:1.47458	test-rmse:2.00696
[369]	train-rmse:1.47374	test-rmse:2.00658
[370]	train-rmse:1.47288	test-rmse:2.00713
[371]	train-rmse:1.47169	test-rmse:2.00694
[372]	train-rmse:1.47127	test-rmse:2.00719
[373]	train-rmse:1.46982	test-rmse:2.00631
[374]	train-rmse:1.46956	test-rmse:2.00607
[375]	train-rmse:1.46899	test-rmse:2.0062
[376]	train-rmse:1.46802	test-rmse:2.00591
[377]	train-rmse:1.46702	test-rmse:2.00573
[378]	train-rmse:1.46664	test-rmse:2.00592
[379]	train-rmse:1.46601	test-rmse:2.00637
[380]	train-rmse:1.46565	test-rmse:2.00633
[381]	train-rmse:1.46459	test-rmse:2.00656
[382]	train-rmse:1.46423	test-rmse:2.00643
[383]	train-rmse:1.46301	test-rmse:2.00654
[384]	train-rmse:1.46277	test-rmse:2.00661
[385]	train-rmse:1.46223	test-rmse:2.00653
[386]	train-r

In [62]:
### max_depth
### 3
rmse_arr = {}
for i in range(3, 10):
    v = i 
    xgb_params['max_depth'] = v
    bst = xgb.train(xgb_params, train_df, 524, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./max_depth.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:26.0723	test-rmse:25.8967
[1]	train-rmse:24.9204	test-rmse:24.7681
[2]	train-rmse:23.8211	test-rmse:23.7179
[3]	train-rmse:22.7702	test-rmse:22.6879
[4]	train-rmse:21.765	test-rmse:21.7061
[5]	train-rmse:20.8032	test-rmse:20.7679
[6]	train-rmse:19.8874	test-rmse:19.8653
[7]	train-rmse:19.0128	test-rmse:19.0134
[8]	train-rmse:18.1775	test-rmse:18.2
[9]	train-rmse:17.3814	test-rmse:17.4385
[10]	train-rmse:16.6201	test-rmse:16.6976
[11]	train-rmse:15.8945	test-rmse:15.9899
[12]	train-rmse:15.2012	test-rmse:15.3157
[13]	train-rmse:14.5382	test-rmse:14.669
[14]	train-rmse:13.9064	test-rmse:14.0474
[15]	train-rmse:13.3034	test-rmse:13.4619
[16]	train-rmse:12.7279	test-rmse:12.8977
[17]	train-rmse:12.1771	test-rmse:12.3654
[18]	train-rmse:11.6526	test-rmse:11.8584
[19]	train-rmse:11.1506	test-rmse:11.3716
[20]	train-rmse:10.671	test-rmse:10.9008
[21]	train-rmse:10.2178	test-rmse:10.4623
[22]	train-rmse:9.78222	test-rmse:10.0407
[23]	train-rmse:9.36837	test-rmse:9.64153
[24]	tra

[194]	train-rmse:1.63057	test-rmse:2.02495
[195]	train-rmse:1.62874	test-rmse:2.02428
[196]	train-rmse:1.62729	test-rmse:2.02377
[197]	train-rmse:1.62668	test-rmse:2.02362
[198]	train-rmse:1.62603	test-rmse:2.02286
[199]	train-rmse:1.62526	test-rmse:2.02321
[200]	train-rmse:1.62392	test-rmse:2.02254
[201]	train-rmse:1.62241	test-rmse:2.02279
[202]	train-rmse:1.6217	test-rmse:2.02232
[203]	train-rmse:1.62117	test-rmse:2.02181
[204]	train-rmse:1.62056	test-rmse:2.02226
[205]	train-rmse:1.61942	test-rmse:2.02243
[206]	train-rmse:1.61674	test-rmse:2.02251
[207]	train-rmse:1.61615	test-rmse:2.02261
[208]	train-rmse:1.61475	test-rmse:2.02245
[209]	train-rmse:1.61349	test-rmse:2.02155
[210]	train-rmse:1.61235	test-rmse:2.02215
[211]	train-rmse:1.61156	test-rmse:2.02213
[212]	train-rmse:1.61111	test-rmse:2.02214
[213]	train-rmse:1.61006	test-rmse:2.02251
[214]	train-rmse:1.60901	test-rmse:2.02147
[215]	train-rmse:1.60775	test-rmse:2.02092
[216]	train-rmse:1.60671	test-rmse:2.01927
[217]	train-

[386]	train-rmse:1.46887	test-rmse:1.99679
[387]	train-rmse:1.46807	test-rmse:1.99635
[388]	train-rmse:1.46681	test-rmse:1.99605
[389]	train-rmse:1.46642	test-rmse:1.99562
[390]	train-rmse:1.46528	test-rmse:1.99602
[391]	train-rmse:1.46443	test-rmse:1.99647
[392]	train-rmse:1.46419	test-rmse:1.99626
[393]	train-rmse:1.46368	test-rmse:1.99649
[394]	train-rmse:1.46272	test-rmse:1.99676
[395]	train-rmse:1.46225	test-rmse:1.99677
[396]	train-rmse:1.46117	test-rmse:1.99686
[397]	train-rmse:1.46071	test-rmse:1.99664
[398]	train-rmse:1.45964	test-rmse:1.99661
[399]	train-rmse:1.45919	test-rmse:1.99627
[400]	train-rmse:1.45801	test-rmse:1.99668
[401]	train-rmse:1.45726	test-rmse:1.99546
[402]	train-rmse:1.45706	test-rmse:1.99541
[403]	train-rmse:1.45658	test-rmse:1.99534
[404]	train-rmse:1.45601	test-rmse:1.99543
[405]	train-rmse:1.45536	test-rmse:1.99467
[406]	train-rmse:1.45487	test-rmse:1.99458
[407]	train-rmse:1.45466	test-rmse:1.99461
[408]	train-rmse:1.45434	test-rmse:1.99486
[409]	train

[54]	train-rmse:2.92452	test-rmse:3.4104
[55]	train-rmse:2.84538	test-rmse:3.32397
[56]	train-rmse:2.77057	test-rmse:3.24688
[57]	train-rmse:2.70053	test-rmse:3.1783
[58]	train-rmse:2.63378	test-rmse:3.11431
[59]	train-rmse:2.57085	test-rmse:3.05244
[60]	train-rmse:2.51102	test-rmse:2.9929
[61]	train-rmse:2.45664	test-rmse:2.93652
[62]	train-rmse:2.40464	test-rmse:2.88666
[63]	train-rmse:2.35602	test-rmse:2.8351
[64]	train-rmse:2.31054	test-rmse:2.78803
[65]	train-rmse:2.26825	test-rmse:2.74259
[66]	train-rmse:2.22723	test-rmse:2.70384
[67]	train-rmse:2.18988	test-rmse:2.66591
[68]	train-rmse:2.15521	test-rmse:2.6302
[69]	train-rmse:2.12197	test-rmse:2.59673
[70]	train-rmse:2.09036	test-rmse:2.56159
[71]	train-rmse:2.06345	test-rmse:2.53587
[72]	train-rmse:2.0358	test-rmse:2.51158
[73]	train-rmse:2.00984	test-rmse:2.48336
[74]	train-rmse:1.98536	test-rmse:2.4607
[75]	train-rmse:1.96351	test-rmse:2.43574
[76]	train-rmse:1.94173	test-rmse:2.41381
[77]	train-rmse:1.92214	test-rmse:2.39374

[247]	train-rmse:1.38591	test-rmse:2.02618
[248]	train-rmse:1.38422	test-rmse:2.02656
[249]	train-rmse:1.38234	test-rmse:2.02698
[250]	train-rmse:1.38071	test-rmse:2.02709
[251]	train-rmse:1.37971	test-rmse:2.02682
[252]	train-rmse:1.37886	test-rmse:2.02652
[253]	train-rmse:1.37743	test-rmse:2.02623
[254]	train-rmse:1.37636	test-rmse:2.02595
[255]	train-rmse:1.37509	test-rmse:2.02587
[256]	train-rmse:1.37428	test-rmse:2.02587
[257]	train-rmse:1.37319	test-rmse:2.02505
[258]	train-rmse:1.3718	test-rmse:2.02512
[259]	train-rmse:1.37058	test-rmse:2.02515
[260]	train-rmse:1.36902	test-rmse:2.02405
[261]	train-rmse:1.36775	test-rmse:2.02401
[262]	train-rmse:1.36661	test-rmse:2.02438
[263]	train-rmse:1.36531	test-rmse:2.02351
[264]	train-rmse:1.36348	test-rmse:2.0234
[265]	train-rmse:1.36284	test-rmse:2.0232
[266]	train-rmse:1.36143	test-rmse:2.02323
[267]	train-rmse:1.36027	test-rmse:2.02304
[268]	train-rmse:1.35814	test-rmse:2.02333
[269]	train-rmse:1.35688	test-rmse:2.02335
[270]	train-rm

...

[248]	train-rmse:0.559065	test-rmse:2.05806
[249]	train-rmse:0.55741	test-rmse:2.05837
[250]	train-rmse:0.55561	test-rmse:2.05829
[251]	train-rmse:0.551513	test-rmse:2.05843
[252]	train-rmse:0.548633	test-rmse:2.05857
[253]	train-rmse:0.547778	test-rmse:2.05881
[254]	train-rmse:0.546871	test-rmse:2.05871
[255]	train-rmse:0.544813	test-rmse:2.05841
[256]	train-rmse:0.544144	test-rmse:2.05854
[257]	train-rmse:0.540333	test-rmse:2.05885
[258]	train-rmse:0.535645	test-rmse:2.05875
[259]	train-rmse:0.534749	test-rmse:2.05875
[260]	train-rmse:0.531075	test-rmse:2.05825
[261]	train-rmse:0.528022	test-rmse:2.05835
[262]	train-rmse:0.525201	test-rmse:2.05905
[263]	train-rmse:0.519837	test-rmse:2.0595
[264]	train-rmse:0.516481	test-rmse:2.05948
[265]	train-rmse:0.51348	test-rmse:2.06001
[266]	train-rmse:0.512073	test-rmse:2.06023
[267]	train-rmse:0.510313	test-rmse:2.06052
[268]	train-rmse:0.509881	test-rmse:2.06063
[269]	train-rmse:0.507357	test-rmse:2.06117
[270]	train-rmse:0.506955	test-rmse:

[436]	train-rmse:0.262819	test-rmse:2.07001
[437]	train-rmse:0.261314	test-rmse:2.07004
[438]	train-rmse:0.2608	test-rmse:2.07011
[439]	train-rmse:0.25972	test-rmse:2.07021
[440]	train-rmse:0.258627	test-rmse:2.07006
[441]	train-rmse:0.257627	test-rmse:2.07005
[442]	train-rmse:0.256872	test-rmse:2.07006
[443]	train-rmse:0.256571	test-rmse:2.07013
[444]	train-rmse:0.255979	test-rmse:2.07013
[445]	train-rmse:0.254758	test-rmse:2.07041
[446]	train-rmse:0.254	test-rmse:2.07047
[447]	train-rmse:0.25318	test-rmse:2.07043
[448]	train-rmse:0.252621	test-rmse:2.07052
[449]	train-rmse:0.251796	test-rmse:2.07054
[450]	train-rmse:0.250721	test-rmse:2.07066
[451]	train-rmse:0.250522	test-rmse:2.07066
[452]	train-rmse:0.250081	test-rmse:2.07056
[453]	train-rmse:0.249118	test-rmse:2.0705
[454]	train-rmse:0.248567	test-rmse:2.07047
[455]	train-rmse:0.24779	test-rmse:2.07046
[456]	train-rmse:0.247346	test-rmse:2.07047
[457]	train-rmse:0.246375	test-rmse:2.07041
[458]	train-rmse:0.245677	test-rmse:2.070

[103]	train-rmse:1.11052	test-rmse:2.15709
[104]	train-rmse:1.09805	test-rmse:2.15194
[105]	train-rmse:1.0876	test-rmse:2.14616
[106]	train-rmse:1.07438	test-rmse:2.14194
[107]	train-rmse:1.06572	test-rmse:2.13796
[108]	train-rmse:1.05627	test-rmse:2.13429
[109]	train-rmse:1.04362	test-rmse:2.13064
[110]	train-rmse:1.03697	test-rmse:2.12689
[111]	train-rmse:1.02614	test-rmse:2.12373
[112]	train-rmse:1.015	test-rmse:2.12198
[113]	train-rmse:1.00534	test-rmse:2.12119
[114]	train-rmse:0.998689	test-rmse:2.11909
[115]	train-rmse:0.990465	test-rmse:2.11822
[116]	train-rmse:0.982863	test-rmse:2.117
[117]	train-rmse:0.975454	test-rmse:2.11537
[118]	train-rmse:0.963061	test-rmse:2.11427
[119]	train-rmse:0.958636	test-rmse:2.11116
[120]	train-rmse:0.951066	test-rmse:2.1106
[121]	train-rmse:0.943473	test-rmse:2.10824
[122]	train-rmse:0.937702	test-rmse:2.10689
[123]	train-rmse:0.929259	test-rmse:2.10568
[124]	train-rmse:0.923295	test-rmse:2.10329
[125]	train-rmse:0.917212	test-rmse:2.10256
[126]

[291]	train-rmse:0.354275	test-rmse:2.06208
[292]	train-rmse:0.352723	test-rmse:2.06216
[293]	train-rmse:0.350397	test-rmse:2.06214
[294]	train-rmse:0.348513	test-rmse:2.0622
[295]	train-rmse:0.347947	test-rmse:2.06222
[296]	train-rmse:0.345444	test-rmse:2.06234
[297]	train-rmse:0.343189	test-rmse:2.06245
[298]	train-rmse:0.341844	test-rmse:2.06246
[299]	train-rmse:0.339821	test-rmse:2.06268
[300]	train-rmse:0.337739	test-rmse:2.06307
[301]	train-rmse:0.336163	test-rmse:2.06325
[302]	train-rmse:0.333246	test-rmse:2.06313
[303]	train-rmse:0.330469	test-rmse:2.06354
[304]	train-rmse:0.328146	test-rmse:2.06373
[305]	train-rmse:0.32679	test-rmse:2.06407
[306]	train-rmse:0.325393	test-rmse:2.06429
[307]	train-rmse:0.322991	test-rmse:2.06452
[308]	train-rmse:0.321855	test-rmse:2.06473
[309]	train-rmse:0.319209	test-rmse:2.06483
[310]	train-rmse:0.3171	test-rmse:2.06498
[311]	train-rmse:0.316698	test-rmse:2.06507
[312]	train-rmse:0.31539	test-rmse:2.06489
[313]	train-rmse:0.313916	test-rmse:2

[479]	train-rmse:0.190098	test-rmse:2.072
[480]	train-rmse:0.189651	test-rmse:2.07207
[481]	train-rmse:0.189426	test-rmse:2.07209
[482]	train-rmse:0.189348	test-rmse:2.07214
[483]	train-rmse:0.189179	test-rmse:2.07216
[484]	train-rmse:0.189038	test-rmse:2.07212
[485]	train-rmse:0.188954	test-rmse:2.07226
[486]	train-rmse:0.188551	test-rmse:2.07219
[487]	train-rmse:0.188086	test-rmse:2.07217
[488]	train-rmse:0.187945	test-rmse:2.0722
[489]	train-rmse:0.187674	test-rmse:2.07233
[490]	train-rmse:0.186948	test-rmse:2.07252
[491]	train-rmse:0.186909	test-rmse:2.07252
[492]	train-rmse:0.186763	test-rmse:2.07252
[493]	train-rmse:0.186077	test-rmse:2.07258
[494]	train-rmse:0.18568	test-rmse:2.07251
[495]	train-rmse:0.185147	test-rmse:2.07258
[496]	train-rmse:0.184814	test-rmse:2.07259
[497]	train-rmse:0.184719	test-rmse:2.07265
[498]	train-rmse:0.184634	test-rmse:2.07262
[499]	train-rmse:0.184503	test-rmse:2.07265
[500]	train-rmse:0.184502	test-rmse:2.07264
[501]	train-rmse:0.184502	test-rmse:

In [60]:
### gamma
### 0.15
rmse_arr = {}
for i in range(0, 200, 5):
    v = i / 100
    xgb_params['gamma'] = v
    bst = xgb.train(xgb_params, train_df, 524, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./gamma.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:26.0723	test-rmse:25.8967
[1]	train-rmse:24.9204	test-rmse:24.7681
[2]	train-rmse:23.8211	test-rmse:23.7179
[3]	train-rmse:22.7702	test-rmse:22.6879
[4]	train-rmse:21.765	test-rmse:21.7061
[5]	train-rmse:20.8032	test-rmse:20.7679
[6]	train-rmse:19.8874	test-rmse:19.8653
[7]	train-rmse:19.0128	test-rmse:19.0134
[8]	train-rmse:18.1775	test-rmse:18.2
[9]	train-rmse:17.3814	test-rmse:17.4385
[10]	train-rmse:16.6201	test-rmse:16.6976
[11]	train-rmse:15.8945	test-rmse:15.9899
[12]	train-rmse:15.2012	test-rmse:15.3157
[13]	train-rmse:14.5382	test-rmse:14.669
[14]	train-rmse:13.9064	test-rmse:14.0474
[15]	train-rmse:13.3034	test-rmse:13.4619
[16]	train-rmse:12.7279	test-rmse:12.8977
[17]	train-rmse:12.1771	test-rmse:12.3654
[18]	train-rmse:11.6526	test-rmse:11.8584
[19]	train-rmse:11.1506	test-rmse:11.3716
[20]	train-rmse:10.671	test-rmse:10.9008
[21]	train-rmse:10.2178	test-rmse:10.4623
[22]	train-rmse:9.78222	test-rmse:10.0407
[23]	train-rmse:9.36837	test-rmse:9.64153
[24]	tra

[194]	train-rmse:1.63039	test-rmse:2.02431
[195]	train-rmse:1.62857	test-rmse:2.02364
[196]	train-rmse:1.62711	test-rmse:2.02314
[197]	train-rmse:1.62648	test-rmse:2.02293
[198]	train-rmse:1.62566	test-rmse:2.02204
[199]	train-rmse:1.6249	test-rmse:2.0224
[200]	train-rmse:1.62356	test-rmse:2.02172
[201]	train-rmse:1.62203	test-rmse:2.02198
[202]	train-rmse:1.62132	test-rmse:2.02151
[203]	train-rmse:1.62078	test-rmse:2.02138
[204]	train-rmse:1.62025	test-rmse:2.02059
[205]	train-rmse:1.61913	test-rmse:2.02077
[206]	train-rmse:1.61644	test-rmse:2.02085
[207]	train-rmse:1.61575	test-rmse:2.02068
[208]	train-rmse:1.61433	test-rmse:2.02114
[209]	train-rmse:1.61309	test-rmse:2.02029
[210]	train-rmse:1.61195	test-rmse:2.02089
[211]	train-rmse:1.61117	test-rmse:2.02089
[212]	train-rmse:1.61072	test-rmse:2.02089
[213]	train-rmse:1.60966	test-rmse:2.02126
[214]	train-rmse:1.60862	test-rmse:2.02025
[215]	train-rmse:1.60784	test-rmse:2.02037
[216]	train-rmse:1.60679	test-rmse:2.01873
[217]	train-r

[386]	train-rmse:1.4664	test-rmse:2.00235
[387]	train-rmse:1.46554	test-rmse:2.00286
[388]	train-rmse:1.46423	test-rmse:2.00253
[389]	train-rmse:1.46382	test-rmse:2.00247
[390]	train-rmse:1.4627	test-rmse:2.00286
[391]	train-rmse:1.46179	test-rmse:2.00339
[392]	train-rmse:1.46156	test-rmse:2.00319
[393]	train-rmse:1.46105	test-rmse:2.00332
[394]	train-rmse:1.46006	test-rmse:2.00305
[395]	train-rmse:1.45953	test-rmse:2.00311
[396]	train-rmse:1.45845	test-rmse:2.00317
[397]	train-rmse:1.45788	test-rmse:2.0025
[398]	train-rmse:1.45689	test-rmse:2.00233
[399]	train-rmse:1.45623	test-rmse:2.00203
[400]	train-rmse:1.45511	test-rmse:2.00245
[401]	train-rmse:1.45444	test-rmse:2.00138
[402]	train-rmse:1.45396	test-rmse:2.0015
[403]	train-rmse:1.45285	test-rmse:2.00121
[404]	train-rmse:1.45232	test-rmse:2.00107
[405]	train-rmse:1.45143	test-rmse:2.00138
[406]	train-rmse:1.45091	test-rmse:2.00135
[407]	train-rmse:1.45069	test-rmse:2.00138
[408]	train-rmse:1.45044	test-rmse:2.00139
[409]	train-rms

[54]	train-rmse:2.9792	test-rmse:3.44421
[55]	train-rmse:2.90211	test-rmse:3.3612
[56]	train-rmse:2.82971	test-rmse:3.28426
[57]	train-rmse:2.76184	test-rmse:3.21626
[58]	train-rmse:2.69731	test-rmse:3.14715
[59]	train-rmse:2.63656	test-rmse:3.08322
[60]	train-rmse:2.57969	test-rmse:3.03072
[61]	train-rmse:2.52698	test-rmse:2.97584
[62]	train-rmse:2.47645	test-rmse:2.91623
[63]	train-rmse:2.42966	test-rmse:2.86985
[64]	train-rmse:2.38576	test-rmse:2.82542
[65]	train-rmse:2.34535	test-rmse:2.78601
[66]	train-rmse:2.30671	test-rmse:2.74626
[67]	train-rmse:2.27137	test-rmse:2.71251
[68]	train-rmse:2.23846	test-rmse:2.67566
[69]	train-rmse:2.20757	test-rmse:2.64216
[70]	train-rmse:2.17761	test-rmse:2.60471
[71]	train-rmse:2.15153	test-rmse:2.57625
[72]	train-rmse:2.12568	test-rmse:2.55484
[73]	train-rmse:2.10106	test-rmse:2.5247
[74]	train-rmse:2.07833	test-rmse:2.49828
[75]	train-rmse:2.05778	test-rmse:2.4753
[76]	train-rmse:2.03753	test-rmse:2.44953
[77]	train-rmse:2.01971	test-rmse:2.43

[247]	train-rmse:1.57878	test-rmse:2.0142
[248]	train-rmse:1.57698	test-rmse:2.01461
[249]	train-rmse:1.57568	test-rmse:2.01515
[250]	train-rmse:1.57475	test-rmse:2.01542
[251]	train-rmse:1.57405	test-rmse:2.01488
[252]	train-rmse:1.57308	test-rmse:2.01471
[253]	train-rmse:1.57248	test-rmse:2.0138
[254]	train-rmse:1.57194	test-rmse:2.01307
[255]	train-rmse:1.57108	test-rmse:2.0137
[256]	train-rmse:1.57001	test-rmse:2.01396
[257]	train-rmse:1.56926	test-rmse:2.01277
[258]	train-rmse:1.56893	test-rmse:2.01278
[259]	train-rmse:1.56819	test-rmse:2.01166
[260]	train-rmse:1.56701	test-rmse:2.01036
[261]	train-rmse:1.56579	test-rmse:2.01022
[262]	train-rmse:1.56495	test-rmse:2.01033
[263]	train-rmse:1.56414	test-rmse:2.00884
[264]	train-rmse:1.56328	test-rmse:2.00817
[265]	train-rmse:1.56296	test-rmse:2.00821
[266]	train-rmse:1.56163	test-rmse:2.0087
[267]	train-rmse:1.56085	test-rmse:2.00882
[268]	train-rmse:1.56001	test-rmse:2.00882
[269]	train-rmse:1.55899	test-rmse:2.00813
[270]	train-rms

...

[110]	train-rmse:1.76815	test-rmse:2.12105
[111]	train-rmse:1.76493	test-rmse:2.11718
[112]	train-rmse:1.76098	test-rmse:2.11443
[113]	train-rmse:1.75798	test-rmse:2.11287
[114]	train-rmse:1.7557	test-rmse:2.11082
[115]	train-rmse:1.75296	test-rmse:2.10978
[116]	train-rmse:1.75031	test-rmse:2.10914
[117]	train-rmse:1.74727	test-rmse:2.10438
[118]	train-rmse:1.74456	test-rmse:2.1016
[119]	train-rmse:1.74192	test-rmse:2.09882
[120]	train-rmse:1.73903	test-rmse:2.09816
[121]	train-rmse:1.73688	test-rmse:2.09733
[122]	train-rmse:1.73455	test-rmse:2.09464
[123]	train-rmse:1.73236	test-rmse:2.09046
[124]	train-rmse:1.73053	test-rmse:2.08884
[125]	train-rmse:1.72803	test-rmse:2.08733
[126]	train-rmse:1.72579	test-rmse:2.08514
[127]	train-rmse:1.72292	test-rmse:2.08291
[128]	train-rmse:1.72115	test-rmse:2.07995
[129]	train-rmse:1.71931	test-rmse:2.07703
[130]	train-rmse:1.71798	test-rmse:2.07529
[131]	train-rmse:1.71648	test-rmse:2.07293
[132]	train-rmse:1.71446	test-rmse:2.07234
[133]	train-r

[302]	train-rmse:1.53047	test-rmse:2.01008
[303]	train-rmse:1.52945	test-rmse:2.01014
[304]	train-rmse:1.52848	test-rmse:2.00982
[305]	train-rmse:1.5275	test-rmse:2.00964
[306]	train-rmse:1.52709	test-rmse:2.00934
[307]	train-rmse:1.52662	test-rmse:2.00916
[308]	train-rmse:1.52522	test-rmse:2.00944
[309]	train-rmse:1.5243	test-rmse:2.00926
[310]	train-rmse:1.52381	test-rmse:2.0097
[311]	train-rmse:1.52347	test-rmse:2.0098
[312]	train-rmse:1.52259	test-rmse:2.00944
[313]	train-rmse:1.5216	test-rmse:2.00888
[314]	train-rmse:1.52098	test-rmse:2.00882
[315]	train-rmse:1.5207	test-rmse:2.00886
[316]	train-rmse:1.51944	test-rmse:2.00914
[317]	train-rmse:1.51844	test-rmse:2.00856
[318]	train-rmse:1.51751	test-rmse:2.0088
[319]	train-rmse:1.51676	test-rmse:2.00907
[320]	train-rmse:1.51536	test-rmse:2.00832
[321]	train-rmse:1.51499	test-rmse:2.00774
[322]	train-rmse:1.51424	test-rmse:2.00761
[323]	train-rmse:1.51329	test-rmse:2.007
[324]	train-rmse:1.51227	test-rmse:2.00701
[325]	train-rmse:1.5

[494]	train-rmse:1.39723	test-rmse:2.00536
[495]	train-rmse:1.39637	test-rmse:2.00527
[496]	train-rmse:1.39601	test-rmse:2.00529
[497]	train-rmse:1.39502	test-rmse:2.00535
[498]	train-rmse:1.39471	test-rmse:2.00523
[499]	train-rmse:1.3937	test-rmse:2.00423
[500]	train-rmse:1.39304	test-rmse:2.00396
[501]	train-rmse:1.39261	test-rmse:2.00411
[502]	train-rmse:1.39195	test-rmse:2.00384
[503]	train-rmse:1.39145	test-rmse:2.00413
[504]	train-rmse:1.39091	test-rmse:2.00418
[505]	train-rmse:1.39036	test-rmse:2.00393
[506]	train-rmse:1.38967	test-rmse:2.00401
[507]	train-rmse:1.38862	test-rmse:2.0035
[508]	train-rmse:1.38796	test-rmse:2.00359
[509]	train-rmse:1.38758	test-rmse:2.00273
[510]	train-rmse:1.38694	test-rmse:2.00282
[511]	train-rmse:1.38617	test-rmse:2.00284
[512]	train-rmse:1.38528	test-rmse:2.00279
[513]	train-rmse:1.38494	test-rmse:2.0028
[514]	train-rmse:1.38461	test-rmse:2.00287
[515]	train-rmse:1.3837	test-rmse:2.00279
[516]	train-rmse:1.38301	test-rmse:2.00205
[517]	train-rms

[163]	train-rmse:1.66655	test-rmse:2.04574
[164]	train-rmse:1.6655	test-rmse:2.04528
[165]	train-rmse:1.66383	test-rmse:2.04526
[166]	train-rmse:1.66322	test-rmse:2.04499
[167]	train-rmse:1.66234	test-rmse:2.04514
[168]	train-rmse:1.66094	test-rmse:2.04456
[169]	train-rmse:1.65978	test-rmse:2.04264
[170]	train-rmse:1.65876	test-rmse:2.04228
[171]	train-rmse:1.65725	test-rmse:2.04164
[172]	train-rmse:1.65575	test-rmse:2.03948
[173]	train-rmse:1.65433	test-rmse:2.03946
[174]	train-rmse:1.65304	test-rmse:2.04008
[175]	train-rmse:1.65089	test-rmse:2.04048
[176]	train-rmse:1.64992	test-rmse:2.04034
[177]	train-rmse:1.64809	test-rmse:2.03898
[178]	train-rmse:1.64769	test-rmse:2.03863
[179]	train-rmse:1.64664	test-rmse:2.03657
[180]	train-rmse:1.6456	test-rmse:2.0363
[181]	train-rmse:1.64406	test-rmse:2.03488
[182]	train-rmse:1.64298	test-rmse:2.03423
[183]	train-rmse:1.64199	test-rmse:2.03308
[184]	train-rmse:1.64045	test-rmse:2.03253
[185]	train-rmse:1.63917	test-rmse:2.0324
[186]	train-rms

[355]	train-rmse:1.48658	test-rmse:2.00587
[356]	train-rmse:1.48627	test-rmse:2.00572
[357]	train-rmse:1.48516	test-rmse:2.00596
[358]	train-rmse:1.48489	test-rmse:2.00585
[359]	train-rmse:1.48395	test-rmse:2.00558
[360]	train-rmse:1.48348	test-rmse:2.00542
[361]	train-rmse:1.48246	test-rmse:2.00611
[362]	train-rmse:1.48163	test-rmse:2.00644
[363]	train-rmse:1.48024	test-rmse:2.00687
[364]	train-rmse:1.47968	test-rmse:2.00632
[365]	train-rmse:1.47883	test-rmse:2.00603
[366]	train-rmse:1.47867	test-rmse:2.00613
[367]	train-rmse:1.47799	test-rmse:2.00641
[368]	train-rmse:1.4771	test-rmse:2.0062
[369]	train-rmse:1.47613	test-rmse:2.00604
[370]	train-rmse:1.47575	test-rmse:2.00617
[371]	train-rmse:1.47427	test-rmse:2.00669
[372]	train-rmse:1.47404	test-rmse:2.00678
[373]	train-rmse:1.47277	test-rmse:2.00535
[374]	train-rmse:1.47241	test-rmse:2.00518
[375]	train-rmse:1.47181	test-rmse:2.00517
[376]	train-rmse:1.47129	test-rmse:2.00528
[377]	train-rmse:1.4705	test-rmse:2.0055
[378]	train-rms

In [58]:
### n_estimators
### 524
rmse_arr = {}
for i in range(500, 530):
    v = i 
    bst = xgb.train(xgb_params, train_df, v, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./n_estimators2.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

[0]	train-rmse:26.0723	test-rmse:25.8967
[1]	train-rmse:24.9204	test-rmse:24.7681
[2]	train-rmse:23.8211	test-rmse:23.7179
[3]	train-rmse:22.7702	test-rmse:22.6879
[4]	train-rmse:21.765	test-rmse:21.7061
[5]	train-rmse:20.8032	test-rmse:20.7679
[6]	train-rmse:19.8874	test-rmse:19.8653
[7]	train-rmse:19.0128	test-rmse:19.0134
[8]	train-rmse:18.1775	test-rmse:18.2
[9]	train-rmse:17.3814	test-rmse:17.4385
[10]	train-rmse:16.6201	test-rmse:16.6976
[11]	train-rmse:15.8945	test-rmse:15.9899
[12]	train-rmse:15.2012	test-rmse:15.3157
[13]	train-rmse:14.5382	test-rmse:14.669
[14]	train-rmse:13.9064	test-rmse:14.0474
[15]	train-rmse:13.3034	test-rmse:13.4619
[16]	train-rmse:12.7279	test-rmse:12.8977
[17]	train-rmse:12.1771	test-rmse:12.3654
[18]	train-rmse:11.6526	test-rmse:11.8584
[19]	train-rmse:11.1506	test-rmse:11.3716
[20]	train-rmse:10.671	test-rmse:10.9008
[21]	train-rmse:10.2178	test-rmse:10.4623
[22]	train-rmse:9.78222	test-rmse:10.0407
[23]	train-rmse:9.36837	test-rmse:9.64153
[24]	tra

[194]	train-rmse:1.63057	test-rmse:2.02495
[195]	train-rmse:1.62874	test-rmse:2.02428
[196]	train-rmse:1.62729	test-rmse:2.02377
[197]	train-rmse:1.62668	test-rmse:2.02362
[198]	train-rmse:1.62603	test-rmse:2.02286
[199]	train-rmse:1.62526	test-rmse:2.02321
[200]	train-rmse:1.62392	test-rmse:2.02254
[201]	train-rmse:1.62241	test-rmse:2.02279
[202]	train-rmse:1.6217	test-rmse:2.02232
[203]	train-rmse:1.62117	test-rmse:2.02181
[204]	train-rmse:1.62056	test-rmse:2.02226
[205]	train-rmse:1.61942	test-rmse:2.02243
[206]	train-rmse:1.61674	test-rmse:2.02251
[207]	train-rmse:1.61615	test-rmse:2.02261
[208]	train-rmse:1.61475	test-rmse:2.02245
[209]	train-rmse:1.61349	test-rmse:2.02155
[210]	train-rmse:1.61235	test-rmse:2.02215
[211]	train-rmse:1.61156	test-rmse:2.02213
[212]	train-rmse:1.61111	test-rmse:2.02214
[213]	train-rmse:1.61006	test-rmse:2.02251
[214]	train-rmse:1.60901	test-rmse:2.02147
[215]	train-rmse:1.60775	test-rmse:2.02092
[216]	train-rmse:1.60671	test-rmse:2.01927
[217]	train-

[386]	train-rmse:1.46887	test-rmse:1.99679
[387]	train-rmse:1.46807	test-rmse:1.99635
[388]	train-rmse:1.46681	test-rmse:1.99605
[389]	train-rmse:1.46642	test-rmse:1.99562
[390]	train-rmse:1.46528	test-rmse:1.99602
[391]	train-rmse:1.46443	test-rmse:1.99647
[392]	train-rmse:1.46419	test-rmse:1.99626
[393]	train-rmse:1.46368	test-rmse:1.99649
[394]	train-rmse:1.46272	test-rmse:1.99676
[395]	train-rmse:1.46225	test-rmse:1.99677
[396]	train-rmse:1.46117	test-rmse:1.99686
[397]	train-rmse:1.46071	test-rmse:1.99664
[398]	train-rmse:1.45964	test-rmse:1.99661
[399]	train-rmse:1.45919	test-rmse:1.99627
[400]	train-rmse:1.45801	test-rmse:1.99668
[401]	train-rmse:1.45726	test-rmse:1.99546
[402]	train-rmse:1.45706	test-rmse:1.99541
[403]	train-rmse:1.45658	test-rmse:1.99534
[404]	train-rmse:1.45601	test-rmse:1.99543
[405]	train-rmse:1.45536	test-rmse:1.99467
[406]	train-rmse:1.45487	test-rmse:1.99458
[407]	train-rmse:1.45466	test-rmse:1.99461
[408]	train-rmse:1.45434	test-rmse:1.99486
[409]	train

[79]	train-rmse:1.98792	test-rmse:2.38948
[80]	train-rmse:1.97255	test-rmse:2.37203
[81]	train-rmse:1.95928	test-rmse:2.35718
[82]	train-rmse:1.94702	test-rmse:2.34309
[83]	train-rmse:1.93533	test-rmse:2.33344
[84]	train-rmse:1.92355	test-rmse:2.32274
[85]	train-rmse:1.91318	test-rmse:2.3086
[86]	train-rmse:1.90307	test-rmse:2.29277
[87]	train-rmse:1.89374	test-rmse:2.27966
[88]	train-rmse:1.88433	test-rmse:2.26866
[89]	train-rmse:1.87613	test-rmse:2.25729
[90]	train-rmse:1.86843	test-rmse:2.24644
[91]	train-rmse:1.86159	test-rmse:2.23568
[92]	train-rmse:1.85438	test-rmse:2.22439
[93]	train-rmse:1.84675	test-rmse:2.21499
[94]	train-rmse:1.84053	test-rmse:2.20713
[95]	train-rmse:1.83425	test-rmse:2.20108
[96]	train-rmse:1.82774	test-rmse:2.19539
[97]	train-rmse:1.82209	test-rmse:2.18738
[98]	train-rmse:1.81662	test-rmse:2.18248
[99]	train-rmse:1.81073	test-rmse:2.17392
[100]	train-rmse:1.8067	test-rmse:2.16715
[101]	train-rmse:1.80186	test-rmse:2.16379
[102]	train-rmse:1.79782	test-rmse

[271]	train-rmse:1.5596	test-rmse:2.00764
[272]	train-rmse:1.55884	test-rmse:2.00705
[273]	train-rmse:1.55831	test-rmse:2.00692
[274]	train-rmse:1.55748	test-rmse:2.00769
[275]	train-rmse:1.5557	test-rmse:2.0069
[276]	train-rmse:1.55469	test-rmse:2.00685
[277]	train-rmse:1.55385	test-rmse:2.00661
[278]	train-rmse:1.5525	test-rmse:2.00743
[279]	train-rmse:1.55181	test-rmse:2.00677
[280]	train-rmse:1.55108	test-rmse:2.00595
[281]	train-rmse:1.55005	test-rmse:2.00677
[282]	train-rmse:1.54842	test-rmse:2.0063
[283]	train-rmse:1.54761	test-rmse:2.00574
[284]	train-rmse:1.54701	test-rmse:2.00565
[285]	train-rmse:1.54642	test-rmse:2.00564
[286]	train-rmse:1.54552	test-rmse:2.00557
[287]	train-rmse:1.54371	test-rmse:2.00509
[288]	train-rmse:1.54287	test-rmse:2.00478
[289]	train-rmse:1.54135	test-rmse:2.00452
[290]	train-rmse:1.54098	test-rmse:2.00423
[291]	train-rmse:1.53968	test-rmse:2.0044
[292]	train-rmse:1.53867	test-rmse:2.00505
[293]	train-rmse:1.5378	test-rmse:2.0048
[294]	train-rmse:1.

...

[247]	train-rmse:1.57999	test-rmse:2.01714
[248]	train-rmse:1.5797	test-rmse:2.01737
[249]	train-rmse:1.57841	test-rmse:2.01791
[250]	train-rmse:1.57778	test-rmse:2.0172
[251]	train-rmse:1.57717	test-rmse:2.0168
[252]	train-rmse:1.5762	test-rmse:2.01662
[253]	train-rmse:1.57578	test-rmse:2.01553
[254]	train-rmse:1.57523	test-rmse:2.01479
[255]	train-rmse:1.57393	test-rmse:2.01485
[256]	train-rmse:1.57289	test-rmse:2.01511
[257]	train-rmse:1.57217	test-rmse:2.01471
[258]	train-rmse:1.57079	test-rmse:2.01462
[259]	train-rmse:1.57004	test-rmse:2.01338
[260]	train-rmse:1.56885	test-rmse:2.01201
[261]	train-rmse:1.56799	test-rmse:2.01203
[262]	train-rmse:1.56717	test-rmse:2.01214
[263]	train-rmse:1.56627	test-rmse:2.01073
[264]	train-rmse:1.56549	test-rmse:2.01005
[265]	train-rmse:1.56469	test-rmse:2.01034
[266]	train-rmse:1.56369	test-rmse:2.00926
[267]	train-rmse:1.56294	test-rmse:2.00863
[268]	train-rmse:1.56223	test-rmse:2.00728
[269]	train-rmse:1.56131	test-rmse:2.0068
[270]	train-rmse

[439]	train-rmse:1.43197	test-rmse:1.99548
[440]	train-rmse:1.43177	test-rmse:1.99542
[441]	train-rmse:1.43087	test-rmse:1.99531
[442]	train-rmse:1.43001	test-rmse:1.99544
[443]	train-rmse:1.42923	test-rmse:1.99564
[444]	train-rmse:1.4289	test-rmse:1.99559
[445]	train-rmse:1.42855	test-rmse:1.99541
[446]	train-rmse:1.42802	test-rmse:1.99528
[447]	train-rmse:1.42776	test-rmse:1.99533
[448]	train-rmse:1.42736	test-rmse:1.995
[449]	train-rmse:1.42679	test-rmse:1.99331
[450]	train-rmse:1.42654	test-rmse:1.99331
[451]	train-rmse:1.42568	test-rmse:1.99379
[452]	train-rmse:1.42528	test-rmse:1.99349
[453]	train-rmse:1.4243	test-rmse:1.99291
[454]	train-rmse:1.42345	test-rmse:1.99281
[455]	train-rmse:1.42271	test-rmse:1.99354
[456]	train-rmse:1.4224	test-rmse:1.99358
[457]	train-rmse:1.42186	test-rmse:1.99388
[458]	train-rmse:1.42126	test-rmse:1.99414
[459]	train-rmse:1.42088	test-rmse:1.99449
[460]	train-rmse:1.42041	test-rmse:1.99408
[461]	train-rmse:1.42001	test-rmse:1.99418
[462]	train-rmse

[104]	train-rmse:1.78915	test-rmse:2.15284
[105]	train-rmse:1.78455	test-rmse:2.1458
[106]	train-rmse:1.78037	test-rmse:2.13897
[107]	train-rmse:1.77652	test-rmse:2.13669
[108]	train-rmse:1.77358	test-rmse:2.13345
[109]	train-rmse:1.77	test-rmse:2.13074
[110]	train-rmse:1.76735	test-rmse:2.12509
[111]	train-rmse:1.76415	test-rmse:2.12048
[112]	train-rmse:1.76019	test-rmse:2.11726
[113]	train-rmse:1.7574	test-rmse:2.1163
[114]	train-rmse:1.7551	test-rmse:2.11448
[115]	train-rmse:1.75239	test-rmse:2.11312
[116]	train-rmse:1.74979	test-rmse:2.11261
[117]	train-rmse:1.74688	test-rmse:2.10821
[118]	train-rmse:1.74396	test-rmse:2.10473
[119]	train-rmse:1.74144	test-rmse:2.10232
[120]	train-rmse:1.73865	test-rmse:2.10169
[121]	train-rmse:1.73632	test-rmse:2.1006
[122]	train-rmse:1.73388	test-rmse:2.09801
[123]	train-rmse:1.7316	test-rmse:2.09379
[124]	train-rmse:1.72979	test-rmse:2.09307
[125]	train-rmse:1.72801	test-rmse:2.09176
[126]	train-rmse:1.7259	test-rmse:2.08971
[127]	train-rmse:1.72

[296]	train-rmse:1.53484	test-rmse:2.00264
[297]	train-rmse:1.53386	test-rmse:2.00236
[298]	train-rmse:1.53329	test-rmse:2.00218
[299]	train-rmse:1.53248	test-rmse:2.00216
[300]	train-rmse:1.53182	test-rmse:2.0027
[301]	train-rmse:1.53133	test-rmse:2.00329
[302]	train-rmse:1.53083	test-rmse:2.00328
[303]	train-rmse:1.53023	test-rmse:2.00382
[304]	train-rmse:1.5292	test-rmse:2.00347
[305]	train-rmse:1.52816	test-rmse:2.0035
[306]	train-rmse:1.52766	test-rmse:2.00382
[307]	train-rmse:1.52724	test-rmse:2.00397
[308]	train-rmse:1.52585	test-rmse:2.00426
[309]	train-rmse:1.52495	test-rmse:2.00415
[310]	train-rmse:1.52442	test-rmse:2.00464
[311]	train-rmse:1.52398	test-rmse:2.00458
[312]	train-rmse:1.52313	test-rmse:2.00428
[313]	train-rmse:1.52219	test-rmse:2.00377
[314]	train-rmse:1.52157	test-rmse:2.00331
[315]	train-rmse:1.52126	test-rmse:2.00319
[316]	train-rmse:1.51997	test-rmse:2.00282
[317]	train-rmse:1.51895	test-rmse:2.00228
[318]	train-rmse:1.51852	test-rmse:2.00244
[319]	train-rm

[488]	train-rmse:1.4034	test-rmse:1.99435
[489]	train-rmse:1.40297	test-rmse:1.99404
[490]	train-rmse:1.40193	test-rmse:1.99365
[491]	train-rmse:1.40149	test-rmse:1.99379
[492]	train-rmse:1.401	test-rmse:1.99411
[493]	train-rmse:1.40053	test-rmse:1.99393
[494]	train-rmse:1.40008	test-rmse:1.99411
[495]	train-rmse:1.39918	test-rmse:1.9938
[496]	train-rmse:1.39884	test-rmse:1.99378
[497]	train-rmse:1.39806	test-rmse:1.99384
[498]	train-rmse:1.39766	test-rmse:1.99404
[499]	train-rmse:1.39663	test-rmse:1.993
[500]	train-rmse:1.39626	test-rmse:1.99307
[501]	train-rmse:1.3958	test-rmse:1.99321
[502]	train-rmse:1.39518	test-rmse:1.99279
[503]	train-rmse:1.39484	test-rmse:1.99299
[504]	train-rmse:1.39403	test-rmse:1.99318
[505]	train-rmse:1.39336	test-rmse:1.99338
[506]	train-rmse:1.39237	test-rmse:1.99326
[507]	train-rmse:1.39177	test-rmse:1.9923
[508]	train-rmse:1.39129	test-rmse:1.99205
[509]	train-rmse:1.39101	test-rmse:1.99216
[510]	train-rmse:1.38998	test-rmse:1.99215
[511]	train-rmse:1.

In [ ]:
### eta
### 0.045
rmse_arr = {}
for i in range(40, 55):
    v = i / 1000
    xgb_params['eta'] = v
    xgb_params['verbosity'] = 0
    bst = xgb.train(xgb_params, train_df, 315, evals=evals)
    ###
    pred = bst.predict(test_df)
    evaluate(data_test, pred)
    x = evaluate(data_test, pred, 0)
    rmse_arr[v] = x

print(f'best_rmse: {min(list(rmse_arr.values()))}')
gen_line_plotly(
    img_path='./eta.html',
    df={'a': list(rmse_arr.values())},
    attrs=list(rmse_arr.keys()),
    auto_open=1,
)

In [50]:
# origin
bst = xgb.train(xgb_params, train_df, 315, evals=evals)
###
pred = bst.predict(test_df)
evaluate(data_test, pred)

[0]	train-rmse:25.777	test-rmse:25.6085
[1]	train-rmse:24.3601	test-rmse:24.2205
[2]	train-rmse:23.0235	test-rmse:22.9095
[3]	train-rmse:21.7603	test-rmse:21.7033
[4]	train-rmse:20.5663	test-rmse:20.5375
[5]	train-rmse:19.4371	test-rmse:19.4368
[6]	train-rmse:18.3746	test-rmse:18.3879
[7]	train-rmse:17.3714	test-rmse:17.4129
[8]	train-rmse:16.4247	test-rmse:16.4918
[9]	train-rmse:15.5328	test-rmse:15.6396
[10]	train-rmse:14.6887	test-rmse:14.8177
[11]	train-rmse:13.8945	test-rmse:14.0491
[12]	train-rmse:13.1455	test-rmse:13.3219
[13]	train-rmse:12.4374	test-rmse:12.6319
[14]	train-rmse:11.7702	test-rmse:11.9722
[15]	train-rmse:11.1418	test-rmse:11.3645
[16]	train-rmse:10.5493	test-rmse:10.7818
[17]	train-rmse:9.98714	test-rmse:10.2363
[18]	train-rmse:9.45861	test-rmse:9.7245
[19]	train-rmse:8.95995	test-rmse:9.23847
[20]	train-rmse:8.49111	test-rmse:8.78191
[21]	train-rmse:8.05344	test-rmse:8.35261
[22]	train-rmse:7.63709	test-rmse:7.94771
[23]	train-rmse:7.2474	test-rmse:7.5701
[24]	t

[195]	train-rmse:1.58509	test-rmse:2.02138
[196]	train-rmse:1.58373	test-rmse:2.0209
[197]	train-rmse:1.58314	test-rmse:2.02035
[198]	train-rmse:1.58226	test-rmse:2.01944
[199]	train-rmse:1.58151	test-rmse:2.01983
[200]	train-rmse:1.57997	test-rmse:2.01909
[201]	train-rmse:1.57855	test-rmse:2.01921
[202]	train-rmse:1.57785	test-rmse:2.01938
[203]	train-rmse:1.57735	test-rmse:2.01935
[204]	train-rmse:1.57547	test-rmse:2.0183
[205]	train-rmse:1.5746	test-rmse:2.01757
[206]	train-rmse:1.57199	test-rmse:2.01771
[207]	train-rmse:1.5714	test-rmse:2.01752
[208]	train-rmse:1.56965	test-rmse:2.01831
[209]	train-rmse:1.56852	test-rmse:2.01666
[210]	train-rmse:1.567	test-rmse:2.01524
[211]	train-rmse:1.56611	test-rmse:2.01519
[212]	train-rmse:1.56484	test-rmse:2.01499
[213]	train-rmse:1.56355	test-rmse:2.01519
[214]	train-rmse:1.56234	test-rmse:2.01529
[215]	train-rmse:1.5619	test-rmse:2.01506
[216]	train-rmse:1.56094	test-rmse:2.0133
[217]	train-rmse:1.56007	test-rmse:2.01331
[218]	train-rmse:1.

2.011719307974481